In [120]:
# DEVOIR 2
# PARTIE III: PRÉDICTIONS
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from itertools import combinations

In [105]:
# RÉGRESSION LINÉAIRE
# On mesure la qualité de la régression linéaire à l'aide du risque empirique
# càd la moyenne des risques (erreur quadratique). Le modèle de meilleure qualité
# lorsque son risque empirique est le plus petit.

# On standardise les données pour comparer les différents modèles.

dataB = pd.read_csv('dataB.csv', index_col=0)
dataB = dataB.values
#dataB = StandardScaler().fit_transform(dataB.values)

In [106]:
def risk_regression(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/5, random_state = 0)

    model = LinearRegression()
    model.fit(x_train, y_train)

    y_predict = model.predict(x_test)

    risk = mean_squared_error(y_test, y_predict)
    return risk

In [107]:
# a) Qualité des prédicteurs en fonction des autres colonnes
# Exemple pour une seule colonne

x = dataB[:,1:]
y = dataB[:,0]
risk_regression(x,y)

0.03537403119239385

In [112]:
# b) Paire de colonnes donnant les meilleures prédictions pour une colonne
# Exemple pour une seule colonne (0)

other_columns = np.delete(np.arange(40), 0)
pairs = list(combinations(other_columns, 2))
risk_array_b = np.zeros(len(pairs))

# Pour chaque paire
for i in range(len(pairs)):
    x = dataB[:,pairs[i]]
    y = dataB[:,0]
    risk_array_b[i] = risk_regression(x,y)

# Trouver la paire de colonnes qui minimisent le risque empirique
min_index = np.argmin(risk_array_b)
min_pairs = np.array(pairs[min_index])
min_pairs

array([16, 18])

In [115]:
# c) Paire de colonnes donnant le meilleur prédicteur pour chacune des caractéristiques en moyenne
pairs = list(combinations(np.arange(40),2))
risk_pairs = np.zeros(len(pairs))

# Supposons que nous avons calculé la qualité en moyenne des 38 autres features pour (0,1):
predicted_columns = np.delete(np.arange(40), pairs[0])
risk_array_c = np.zeros(predicted_columns.shape[0])

for i in range(predicted_columns.shape[0]):
    x = dataB[:,pairs[0]]
    y = dataB[:,i]
    risk_array_c[i] = risk_regression(x,y)

# np.mean(risk_array) se trouve dans risk_pairs
# Retourne opt_index qui est l'argmin de risk_array
# Réponse: optimal_pair = np.array(pairs[opt_index])
print(np.mean(risk_array))

0.3241724334608466


In [119]:
# CLASSIFIEUR BAYÉSIEN
dataC = pd.read_csv('dataC.csv', index_col=0)
dataC = dataC.values
dataC

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [131]:
# TEST sur prédire feature 0

x = dataC[:,1:]
y = dataC[:,0]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/5, random_state = 0)

model = MultinomialNB()
model.fit(x_train, y_train)

y_predict = model.predict(x_test)
error = 1 - accuracy_score(y_test, y_predict)
error

0.125